# Initializing the SysML v2 API

In [2]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json

#host = "<SysML2-API-Repo-Host:Port>"
host = "Specify the URL of the SysML v2 API and Service Repository"

# Get projects

In [3]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    
    df = pd.DataFrame({'Project Name':[], 'Project ID':[]})
    for p in projects:
        df = pd.concat([df, pd.DataFrame({'Project Name':[p['name']], 'Project ID':[p['@id']]})], ignore_index=True)
    display(df)

,Project Name,Project ID
0,7b-Variant Configurations Mon Mar 13 17:54:29 ...,000e9890-6935-43e6-a5d7-5d7cac601f4c
1,6-Individual and Snapshots Mon Mar 13 17:53:49...,00bbbdc9-ea5e-4ab6-8198-94b4708639c4
2,11a-View-Viewpoint Mon Mar 13 17:59:02 EDT 2023,021b60e9-1d9d-4c0e-af6b-8478b5928ccf
3,VehiclePackage Fri May 05 12:28:40 UTC 2023,0768a0aa-9be9-457d-b693-12d45c68343c
4,sleon_filtering_test Sat Jun 10 03:30:40 UTC 2023,0a8f97fe-a4c8-4806-bfa9-0d95e377761a
...,...,...
71,updated-test-project-19/05/2023 13:36:25,f5d672c5-8a36-4a5a-8fcc-4eb7bfb703ff
72,13b-Safety and Security Features Element Group...,f9079904-26cf-437e-ab74-e1bd6b6013fb
73,ts-usage-views Fri Jun 09 06:35:30 UTC 2023,fcf4f295-8813-4737-ad97-f21267e3738b
74,VehicleMasses_MapleConnector Mon Jan 22 20:32:...,fee1b365-a84d-4de1-ba56-1d364f10478c


# Define a function to recursively iterate over feature members 
## Recursive graph traversal using FeatureMembership relations

In [34]:
def get_member_features(project_id, commit_id, element_id, member_type, indent):
    
    # Fetch the element
    element_url = f"{host}/projects/{project_id}/commits/{commit_id}/elements/{element_id}" 
    response = requests.get(element_url)
    
    element_data = response.json()
    element_name = element_data['name']
    element_id = element_data['@id']
    element_type = element_data ['@type']
    
    if element_type == member_type:
        print(f"{indent} - {element_name} (id = {element_id}, type = {element_type})")
        # Feature memberships
        element_features = element_data['feature']
        if len(element_features) > 0:
            for feature in element_features:
                get_member_features(project_id, commit_id, feature['@id'], member_type, indent + "  ")


## Project

In [35]:
spacecraft_project_id = 'a24a3f81-db33-4514-8cee-11b8db59d3e8'

In [36]:
spacecraft_project_commit_id = ''

commits_url = f"{host}/projects/{spacecraft_project_id}/commits" 
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    commits = commits_response.json()
    pprint(commits)
    spacecraft_project_commit_id = commits[0]['@id']


[{'@id': '40666fb1-cab2-415e-8764-75121b14ec87',
  '@type': 'Commit',
  'created': '2023-06-13T18:54:44.606759-04:00',
  'description': None,
  'owningProject': {'@id': 'a24a3f81-db33-4514-8cee-11b8db59d3e8'},
  'previousCommit': None}]


## Query to find Spacecraft part

In [37]:
 
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'Spacecraft'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle1_id = ''
query_url = f"{host}/projects/{spacecraft_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    spacecraft_part_id = query_response_json[0]['@id']

[{'@id': '6a262fe1-6958-4c05-8ff1-c19d4cd68c41',
  '@type': 'PartUsage',
  'name': 'Spacecraft',
  'owner': {'@id': 'e51d38d7-3459-4c19-b760-1d6a257641b1'}}]


## Parts tree for Spacecraft (recursive FeatureMembership relation navigation)

In [38]:
get_member_features(spacecraft_project_id,spacecraft_project_commit_id, spacecraft_part_id,'PartUsage'," ")

  - Spacecraft (id = 6a262fe1-6958-4c05-8ff1-c19d4cd68c41, type = PartUsage)
    - Payload_Subsystem (id = 091aada9-3687-4bfc-a683-7d8b92972935, type = PartUsage)
      - Payload_sensor (id = 9a2721e1-410c-49f9-9551-9c8721ef589e, type = PartUsage)
      - Payload_software (id = 31fc0cdb-4ac4-4da3-bee8-713b41e5781c, type = PartUsage)
    - Power_Subsystem (id = 32ab132f-9db6-4ea0-9c77-d399fdf41faa, type = PartUsage)
      - Solar_Array (id = 97713133-38b4-42a3-929f-6a1f024ba37d, type = PartUsage)
      - Battery (id = 4518e4c0-a6a5-441e-b007-38b66a4fc589, type = PartUsage)
    - Communications_Subsystem (id = be62763d-fdff-425a-a993-d81cdf5fcf93, type = PartUsage)
      - Communication_Software (id = f88566d8-65cd-4a9e-91df-44eda6faf7db, type = PartUsage)
      - High_Rate_Antenna (id = cf7f9c85-f994-4496-a80e-2517d2bc75c7, type = PartUsage)
    - Thermal_Subsystem (id = 0411b359-503e-43ef-af18-1b904f6b35b2, type = PartUsage)
      - Radiator (id = 87963be9-6564-4e5b-b890-ab195f2faeb4, 

In [39]:
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'Spacecraft_Specification'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'RequirementUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle_specification_id = ''
query_url = f"{host}/projects/{spacecraft_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    spacecraft_specification_id = query_response_json[0]['@id']

[{'@id': '459f5ec9-dc82-45c0-87e3-d013ed6773e5',
  '@type': 'RequirementUsage',
  'name': 'Spacecraft_Specification',
  'owner': {'@id': '26bc955e-5ea4-4a51-8099-db42169e7436'}}]


In [40]:
get_member_features(spacecraft_project_id, spacecraft_project_commit_id, spacecraft_specification_id,"RequirementUsage"," ")

  - Spacecraft_Specification (id = 459f5ec9-dc82-45c0-87e3-d013ed6773e5, type = RequirementUsage)
    - Launch_Vehicle_Interface (id = 9df26fa1-e5c6-4679-8c45-4c482b544497, type = RequirementUsage)
      - Launch_Vehicle_Mechanical_Interface (id = 6a262fdd-1964-490b-ad2b-b8f444aa60d7, type = RequirementUsage)
      - Launch_Vehicle_Electrics_Interface (id = 0674630a-ee6e-4e03-8ada-3d24cc82f700, type = RequirementUsage)
    - Functional_and_Performance (id = b2a2a88e-d6c8-4277-a43c-ce9c7e74bf95, type = RequirementUsage)
      - Probability_of_Detection (id = b896853b-cdc3-4776-b28e-b11dba0701f7, type = RequirementUsage)
      - Probability_of_False_Alarm (id = d1941cc0-fd36-4a74-933c-cc444e105d5d, type = RequirementUsage)
      - Pointing_Accuracy (id = 24f75812-2686-4445-9c48-dd842b0a0b17, type = RequirementUsage)
      - Downlink_Capacity (id = 71d84b57-0033-4a20-a598-781d547d5c51, type = RequirementUsage)
      - Data_Storage (id = 9d9bcba4-4d1c-421a-b947-ebd1d19fc931, type = Require